# Webshop - Transaktionen und Integrität - Übungen

In [ ]:
# verwendete Pakete importieren
import mysql.connector
import sshtunnel
import time

### Herstellen der Datenbankverbindung

#### Notwendige Verbindungsparameter

In [ ]:
private_key_ssh = './key'  # Keyfile für die SSH-Verbindung muss im selben Ordner liegen (ILIAS-Materialordner "Infos zur Übungsdatenbank")
username_ssh = ''  # (1) Username für die SSH-Verbindung (ILIAS - Schritt (1) )
password_ssh = ''  # (2) Passwort für die SSH-Verbindung (ILIAS - Schritt (2) )
host_ssh = ''  # (3) die IP-Adresse der Übungsdatenbank für die SSH-Verbindung - (ILIAS - Schritt (3) )


username_db = ''  # (4) Username für die Datenbankverbindung (ILIAS - Schritt (4) )
password_db = ''  # (5) Passwort für die Datenbankverbindung (ILIAS - Schritt (5) )
database = ''  # (6) Ihre Übungsdatenbank (ILIAS - Schritt (6) )
hostname_db = 'localhost'  # wir nutzen localhost für die Datenbank (wegen der SSH-Verbindung)

#### Aufbauen des SSH-Tunnels

In [ ]:
tunnel = sshtunnel.SSHTunnelForwarder(
        (host_ssh, 22),
        ssh_username=username_ssh,
        ssh_pkey=private_key_ssh,
        ssh_private_key_password=password_ssh,
        remote_bind_address=(hostname_db, 3306),
        local_bind_address=("0.0.0.0", 3306)
)

## Beispielkunde und Bearbeiter

In [ ]:
# Beispieldaten
kunde = 'lea.schmidt@xy.de'
bearbeiter = 'julia.meier@ab.de'
bestellstatus = 1 # Bestellung erhält status "bestellt"

## Warenkorb vorbereiten

In [ ]:
with tunnel:
    # Verbindung aufbauen
    conn = mysql.connector.connect(host=hostname_db, user=username_db, passwd=password_db, autocommit=True, database=database, auth_plugin='mysql_native_password')
    cur = conn.cursor()
    
    try:
        # beispielhaften Warenkorb anlegen
        cur.execute(f"delete from warenkorbposition where kunde_email = '{kunde}'")  # Warenkorb leeren
        cur.execute(f"insert into warenkorbposition values ('{kunde}', 1, 100), ('{kunde}', 2, 200), ('{kunde}', 3, 300), ('{kunde}', 4, 400)")  # Warenkorb füllen
        
    except Exception as e:
        print(f"{e}")    
    
    conn.close()   

## Warenkorb in Bestellung überführen

In [ ]:
with tunnel:
    # Verbindung aufbauen
    conn = mysql.connector.connect(host=hostname_db, user=username_db, passwd=password_db, autocommit=False, database=database, auth_plugin='mysql_native_password')
    cur = conn.cursor()
    
    try:
        # Transaktion starten (alle folgenden Operationen gemeinsam als atomar betrachten)
        conn.start_transaction()
        print("Transaktion 'Warenkorb in Bestellung überführen' begonnen")
        
        # Bestellung mit auto-generierte Bestellnummer anlegen
        sql = f"insert into bestellung (bestelldatum, bearbeitet_email, kunde, statusid) values " \
              f"(now(), '{bearbeiter}', '{kunde}', {bestellstatus})"
        print(f"Führe aus: {sql}")
        cur.execute(sql)
        
        # ID der eingefügten Bestellung holen
        new_order_id = cur.lastrowid
        
        # ALle Positionen aus dem Warenkorb als Bestellpositionen übernehmen
        sql = f"insert into bestellposition " \
              f"select {new_order_id}, pnr, menge from warenkorbposition w where kunde_email = '{kunde}'"
        print(f"Führe aus: {sql}")
        cur.execute(sql)        
        
        time.sleep(0)
        
        # Warenkorbpositionen löschen
        sql = f"delete from warenkorbposition where kunde_em = '{kunde}'"
        print(f"Führe aus: {sql}")
        cur.execute(sql)
        
        # Transaktion abschließen ("commit")
        conn.commit()
        print(f"Transaktion 'Warenkorb in Bestellung überführen' abgeschlossen. Neue Bestellnummer: {new_order_id}")
        
    except Exception as e:
        # Im Fehlerfall die bisherigen Änderungen rückgängig machen ("rollback")
        print(f"Fehler: {e}")
        print(f"Bisherige Änderungen rückgänig machen. Neue Bestellnummer nicht mehr im System: {new_order_id}")
        conn.rollback()
            
    conn.close()
    
    